In [1]:
import os

In [2]:
%pwd

'e:\\Projects\\Unfinished\\End-to-End-Chest-Cancer-Classification-MLFlow-DVC\\research'

In [3]:
os.chdir('../')
%pwd

'e:\\Projects\\Unfinished\\End-to-End-Chest-Cancer-Classification-MLFlow-DVC'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import zipfile
import gdown
from ChestCancerClassification import logger
from ChestCancerClassification.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} to {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} to {zip_download_dir}")
        except Exception as e:
            raise e
        
    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-02-23 12:51:33,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-23 12:51:33,014: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-23 12:51:33,015: INFO: common: created directory at: artifacts]
[2025-02-23 12:51:33,016: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-23 12:51:33,017: INFO: 4019176311: Downloading data from https://drive.google.com/file/d/1bnvvj1y6Rr1_HZc3-QhWC596hWYR8kCj/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1bnvvj1y6Rr1_HZc3-QhWC596hWYR8kCj
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1bnvvj1y6Rr1_HZc3-QhWC596hWYR8kCj&confirm=t&uuid=33b2ff50-2319-4ba2-b5b0-076c34d41d1b
To: e:\Projects\Unfinished\End-to-End-Chest-Cancer-Classification-MLFlow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:07<00:00, 6.75MB/s]


[2025-02-23 12:51:47,050: INFO: 4019176311: Downloaded data from https://drive.google.com/file/d/1bnvvj1y6Rr1_HZc3-QhWC596hWYR8kCj/view?usp=sharing to artifacts/data_ingestion/data.zip]
